In [1]:
import pandas as pd
import plotly
import os
import sys
import numpy as np
import pickle
import re

In [39]:
CLUSTERS_FOLDER="/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/clusters_0.1_embeddings_q0.001_transformer_80_double_n_pair_margin_pvalue_0.6"
CLUSTERS_IDENTIFICATIONS_FOLDER="/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/identifications_q0.001_transformer_80_double_n_pair_margin_pvalue_0.6"
ANALYSIS_OUTPUT_FOLDER = "/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/analysis"

In [59]:
models=[
    {
        'clusters':"/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/clusters_0.1_embeddings_q0.001_transformer_80_double_n_pair_margin_pvalue_0.6",
        'identifications':"/media/eduseiti/data_storage_1TB/unicamp/clustering_linfeng_sample_pvalues/identifications/identifications_q0.001_transformer_80_double_n_pair_margin_pvalue_0.6"
    },
    {
        'clusters':"/media/eduseiti/data_storage_1TB/unicamp/clustering_linfeng_sample_pvalues/clusters/clusters_0.1_embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6",
        'identifications':"/media/eduseiti/data_storage_1TB/unicamp/clustering_linfeng_sample_pvalues/identifications/identifications_q0.001_transformer_64_double_n_pair_pvalue_0.6"
    },
    {
        'clusters':"/media/eduseiti/Seagate_Expansion_Drive1/unicamp/clustering/linfeng/sample/backup_first_comparisson_20200225/clusters/clusters_sample_0.1_maracluster_verification",
        'identifications':"/media/eduseiti/Seagate_Expansion_Drive1/unicamp/clustering/linfeng/sample/backup_first_comparisson_20200225/consensus_identifications/identifications_maracluster"
    },
    {
        'clusters':"/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/clusters_0.1_embeddings_q0.001_all_lstm80_3layer_double_n_pair_margin_0.7_pvalue_0.6",
        'identifications':"/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/identifications_q0.001_all_lstm80_3layer_double_n_pair_margin_0.7_pvalue_0.6"
    },
]

In [4]:
ALL_IDENTIFICATIONS = "/media/eduseiti/Seagate_Expansion_Drive1/unicamp/clustering/linfeng/sample/backup_first_comparisson_20200225/identifications_sample_0.1_nterm_fix/sample_experiment_identifications/percolator.target.psms.txt"

CLUSTERED_FILES_LIST = "/media/eduseiti/bigdata02/unicamp/doutorado/clustering/linfeng/sample/sample_0.1_files.txt"

Q_VALUE=float("0.001")


CLUSTERS_CONSENSUS_IDENTIFICATIONS_FILENAME_SUFFIX="_identifications.tsv"

SEQUENCES_CLUSTERS_FILENAME_SUFFIX="_q{}_sequences_clusters.pkl"
CLUSTERS_SPECTRA_FILENAME_SUFFIX="_q{}_clusters_spectra.pkl"
CLUSTER_ANALYSIS_FILENAME_SUFFIX="_q{}_clusters_analysis.tsv"

In [5]:
#
# For each cluster, analyzes its spectra identification
#

def identify_clusters_spectra_fast(which_clusters, identifications):

    clusters_spectra = {}

    sequences_clusters = {}
    
    
    how_many_not_identified = 0
    
    ids_index = 0
    
    max_ids_index = identifications.shape[0]

    #
    # Transform the dataframes into numpy arrays
    #

    clusters_array = which_clusters.to_numpy()
    identifications_array = identifications.to_numpy()
    
    #
    # Get required columns indexes
    #

    CLUSTERS_FILE_IDX = which_clusters.columns.get_loc("file_idx")
    CLUSTERS_SCAN = which_clusters.columns.get_loc("scan")
    CLUSTERS_CLUSTER = which_clusters.columns.get_loc("cluster")

    IDS_FILE_IDX = identifications.columns.get_loc("file_idx")
    IDX_SCAN = identifications.columns.get_loc("scan")
    IDX_QVALUE = identifications.columns.get_loc("percolator q-value")
    IDX_SEQUENCE = identifications.columns.get_loc("sequence")


    #
    # Loop through all the clusters, and creates a dictionary with the identification of the spectra included in each one of them
    #

    for index in range(len(clusters_array)):
        
        new = {}
        
        while (ids_index < max_ids_index) and (identifications_array[ids_index][IDS_FILE_IDX] < clusters_array[index][CLUSTERS_FILE_IDX]):
            ids_index += 1
            
        if (ids_index < max_ids_index) and identifications_array[ids_index][IDS_FILE_IDX] == clusters_array[index][CLUSTERS_FILE_IDX]:
            while (ids_index < max_ids_index) and (identifications_array[ids_index][IDX_SCAN] < clusters_array[index][CLUSTERS_SCAN]):
                ids_index += 1
                
            if (ids_index < max_ids_index) and identifications_array[ids_index][IDX_SCAN] == clusters_array[index][CLUSTERS_SCAN]:
                new = {"sequence":identifications_array[ids_index][IDX_SEQUENCE], 
                       "q-value":identifications_array[ids_index][IDX_QVALUE], 
                       "file":identifications_array[ids_index][IDS_FILE_IDX],
                       "scan":identifications_array[ids_index][IDX_SCAN]}

                ids_index += 1
            
        if len(new) == 0:
            new = {"sequence":'not identified', 
                   "q-value":1.0, 
                   "file":clusters_array[index][CLUSTERS_FILE_IDX],
                   "scan":clusters_array[index][CLUSTERS_SCAN]}
            
            how_many_not_identified += 1

            
        if new['sequence'] in sequences_clusters:
            sequences_clusters[new['sequence']].append(clusters_array[index][CLUSTERS_CLUSTER])
        else:
            sequences_clusters[new['sequence']] = [clusters_array[index][CLUSTERS_CLUSTER]]
        
                        
        if clusters_array[index][CLUSTERS_CLUSTER] in clusters_spectra:
            clusters_spectra[clusters_array[index][CLUSTERS_CLUSTER]]['spectra'].append(new)
            
            if new['sequence'] not in clusters_spectra[clusters_array[index][CLUSTERS_CLUSTER]]['sequences']:
                clusters_spectra[clusters_array[index][CLUSTERS_CLUSTER]]['sequences'].append(new['sequence'])
        else:
            clusters_spectra[clusters_array[index][CLUSTERS_CLUSTER]] = {'spectra':[new], 'sequences':[new['sequence']]}
            
    return clusters_spectra, sequences_clusters


In [6]:
def fileindex(row, file_index, spectra_files):
    file_index[row.name] = spectra_files[row['file'].split("/")[-1]]  

In [41]:
def analize_cluster(cluster_file, cluster_consensus_ids_file, spectra_files_dict, all_ids, q_value, analysis_output_file_root, save_data=False):   
    

    clusters = pd.read_csv(cluster_file, names=["file", "scan", "cluster"], sep="\t")

    #
    # Observations on clusters file:
    #
    # - Clusters index starts in 1
    # - Scan index starts in 0
    #

    consensus_ids = pd.read_csv(cluster_consensus_ids_file, sep="\t")

    

    #
    # Create a file index (file_idx) column in clusters dataframe
    #

    file_index = np.zeros(clusters.shape[0])

    clusters.apply(fileindex, axis=1, args=(file_index, spectra_files_dict))

    clusters['file_idx'] = file_index.astype(int)            


    #
    # Sort dataframe to speedup processing
    #

    clusters = clusters.sort_values(['file_idx', 'scan'])

    cluster_ids, sequences_clusters = identify_clusters_spectra_fast(clusters, all_ids)

    sequences_purity = {}

    total_spectra = 0

    unique_count = 0
    unique_and_cluster_pure_count = 0

    for sequence in sequences_clusters:

        total_spectra += len(sequences_clusters[sequence])

        unique_clusters = np.unique(sequences_clusters[sequence])

        different_clusters = len(unique_clusters)

        sequences_purity[sequence] = unique_clusters

        if different_clusters == 1:
            unique_count += 1    

            if len(cluster_ids[unique_clusters[0]]['sequences']) == 1:
                unique_and_cluster_pure_count += 1


    if save_data:
        #
        # Save the results
        #

        with open(analysis_output_file_root + SEQUENCES_CLUSTERS_FILENAME_SUFFIX.format(q_value), 'wb') as outputFile:
            pickle.dump(sequences_purity, outputFile, pickle.HIGHEST_PROTOCOL)

        with open(analysis_output_file_root + CLUSTERS_SPECTRA_FILENAME_SUFFIX.format(q_value), 'wb') as outputFile:
            pickle.dump(cluster_ids, outputFile, pickle.HIGHEST_PROTOCOL)    

            
    #
    # Determine the number of cluster with more than a single identified sequence
    #

    sequences_count = []

    for key, cluster in cluster_ids.items():

        if "not identified" in cluster['sequences']:
            sequences_count.append(len(cluster['sequences']) - 1)
        else:
            sequences_count.append(len(cluster['sequences']))    
    
    sequences_count = np.array(sequences_count)
    
    clusters_count_more_than_one_sequence = sequences_count[sequences_count > 1]

    return {'unique_and_cluster_pure_count':unique_and_cluster_pure_count, 
            'percent_total_unique':unique_and_cluster_pure_count / unique_count,
            'unique_count': unique_count, 
            'percent_total_sequences_clusters': unique_count / (len(sequences_clusters) - 1),
            'num_sequences_clusters':len(sequences_clusters) - 1,
            'num_clusters_only_not_identified':len(sequences_count[sequences_count == 0]),
            'num_clusters_single_sequence':len(sequences_count[sequences_count == 1]),
            'num_clusters_multiple_sequences':len(clusters_count_more_than_one_sequence),
            'percent_total_clusters':len(clusters_count_more_than_one_sequence) / len(cluster_ids),
            'num_total_clusters':len(cluster_ids),
            'num_not_identified_spectra':len(sequences_clusters['not identified']),
            'percent_total_spectra': len(sequences_clusters['not identified']) / total_spectra,
            'num_total_spectra':total_spectra}, sequences_purity, cluster_ids

In [8]:
def analize_model_clusters(clusters_folder, consensus_identifications_folder, analysis_output_folder, spectra_files, all_ids, q_value):
    
    all_ids = all_ids[all_ids['percolator q-value'] < q_value]
    
    results = []
    
    for root, directories, files in os.walk(clusters_folder):
        for file in sorted(files):
            if re.search(".+_p[0-9\.]+\.tsv$", file):
                print("\nFilename root={}".format(file.split(".tsv")[0]))

                filename_root = file.split(".tsv")[0]
                
                cluster_identification_filename = os.path.join(consensus_identifications_folder, filename_root) + CLUSTERS_CONSENSUS_IDENTIFICATIONS_FILENAME_SUFFIX
                
                if os.path.isfile(cluster_identification_filename):
                    results_cluster, _, _ = analize_cluster(os.path.join(clusters_folder, file), 
                                                            cluster_identification_filename, 
                                                            spectra_files, all_ids, q_value,
                                                            os.path.join(analysis_output_folder, filename_root))

                    results_cluster['cluster'] = file

                    print(results_cluster)

                    results.append(results_cluster)
                else:
                    print("Cluster identification file not found! {}".format(cluster_identification_filename))
                
                
    pd.DataFrame(results).to_csv(os.path.join(analysis_output_folder, clusters_folder.split("/")[-1]) + CLUSTER_ANALYSIS_FILENAME_SUFFIX.format(q_value), index=False, sep='\t')
            

In [9]:
all_ids = pd.read_csv(ALL_IDENTIFICATIONS, sep="\t")

#
# Observations on identifications file:
#
# - File indexes start in 0
# - Scan indexes start in 1
#


#
# Filter only the high quality identifications, and fix the scan indexing to match
# the cluster results
#

all_ids['scan'] = all_ids['scan'] - 1
all_ids = all_ids.sort_values(['file_idx', 'scan'])

In [10]:
spectra_files = {}
spectra_files_count = 0


with open(CLUSTERED_FILES_LIST, "r") as inputFile:
    while True:
        filename = inputFile.readline().strip()

        if filename != "":
            spectra_files[filename.split("/")[-1]] = spectra_files_count
            spectra_files_count += 1
        else:
            break

In [60]:
analize_model_clusters(models[3]['clusters'], models[3]['identifications'], ANALYSIS_OUTPUT_FOLDER, spectra_files, all_ids, 0.01)


Filename root=embeddings_q0.001_all_lstm80_3layer_double_n_pair_margin_0.7_pvalue_0.6.clusters_p100.000000
{'unique_and_cluster_pure_count': 14277, 'percent_total_unique': 1.0, 'unique_count': 14277, 'percent_total_sequences_clusters': 0.2884475513172782, 'num_sequences_clusters': 49496, 'num_clusters_only_not_identified': 601258, 'num_clusters_single_sequence': 310888, 'num_clusters_multiple_sequences': 0, 'percent_total_clusters': 0.0, 'num_total_clusters': 912146, 'num_not_identified_spectra': 601258, 'percent_total_spectra': 0.6591685980095292, 'num_total_spectra': 912146, 'cluster': 'embeddings_q0.001_all_lstm80_3layer_double_n_pair_margin_0.7_pvalue_0.6.clusters_p100.000000.tsv'}

Filename root=embeddings_q0.001_all_lstm80_3layer_double_n_pair_margin_0.7_pvalue_0.6.clusters_p64.506800
{'unique_and_cluster_pure_count': 1139, 'percent_total_unique': 0.03380223171889839, 'unique_count': 33696, 'percent_total_sequences_clusters': 0.6807822854372071, 'num_sequences_clusters': 49496, 

{'unique_and_cluster_pure_count': 18685, 'percent_total_unique': 0.6321469652885852, 'unique_count': 29558, 'percent_total_sequences_clusters': 0.597179570066268, 'num_sequences_clusters': 49496, 'num_clusters_only_not_identified': 214170, 'num_clusters_single_sequence': 87133, 'num_clusters_multiple_sequences': 3953, 'percent_total_clusters': 0.012949786408784757, 'num_total_clusters': 305256, 'num_not_identified_spectra': 601258, 'percent_total_spectra': 0.6591685980095292, 'num_total_spectra': 912146, 'cluster': 'embeddings_q0.001_all_lstm80_3layer_double_n_pair_margin_0.7_pvalue_0.6.clusters_p95.897800.tsv'}

Filename root=embeddings_q0.001_all_lstm80_3layer_double_n_pair_margin_0.7_pvalue_0.6.clusters_p96.000000
{'unique_and_cluster_pure_count': 18842, 'percent_total_unique': 0.6421949556918882, 'unique_count': 29340, 'percent_total_sequences_clusters': 0.5927751737514143, 'num_sequences_clusters': 49496, 'num_clusters_only_not_identified': 219044, 'num_clusters_single_sequence': 

In [58]:
analize_model_clusters(models[0]['clusters'], models[0]['identifications'], ANALYSIS_OUTPUT_FOLDER, spectra_files, all_ids, 0.01)


Filename root=embeddings_q0.001_transformer_80_double_n_pair_margin_pvalue_0.6.clusters_p100.000000
{'unique_and_cluster_pure_count': 14277, 'percent_total_unique': 1.0, 'unique_count': 14277, 'percent_total_sequences_clusters': 0.2884475513172782, 'num_sequences_clusters': 49496, 'num_clusters_only_not_identified': 601258, 'num_clusters_single_sequence': 310888, 'num_clusters_multiple_sequences': 0, 'percent_total_clusters': 0.0, 'num_total_clusters': 912146, 'num_not_identified_spectra': 601258, 'percent_total_spectra': 0.6591685980095292, 'num_total_spectra': 912146, 'cluster': 'embeddings_q0.001_transformer_80_double_n_pair_margin_pvalue_0.6.clusters_p100.000000.tsv'}

Filename root=embeddings_q0.001_transformer_80_double_n_pair_margin_pvalue_0.6.clusters_p64.506800
{'unique_and_cluster_pure_count': 1139, 'percent_total_unique': 0.03341155764153711, 'unique_count': 34090, 'percent_total_sequences_clusters': 0.6887425246484564, 'num_sequences_clusters': 49496, 'num_clusters_only_no

{'unique_and_cluster_pure_count': 20279, 'percent_total_unique': 0.7432288803371816, 'unique_count': 27285, 'percent_total_sequences_clusters': 0.5512566672054308, 'num_sequences_clusters': 49496, 'num_clusters_only_not_identified': 245283, 'num_clusters_single_sequence': 108164, 'num_clusters_multiple_sequences': 1566, 'percent_total_clusters': 0.004411106072172005, 'num_total_clusters': 355013, 'num_not_identified_spectra': 601258, 'percent_total_spectra': 0.6591685980095292, 'num_total_spectra': 912146, 'cluster': 'embeddings_q0.001_transformer_80_double_n_pair_margin_pvalue_0.6.clusters_p95.897800.tsv'}

Filename root=embeddings_q0.001_transformer_80_double_n_pair_margin_pvalue_0.6.clusters_p96.000000
{'unique_and_cluster_pure_count': 20243, 'percent_total_unique': 0.74769151215188, 'unique_count': 27074, 'percent_total_sequences_clusters': 0.54699369646032, 'num_sequences_clusters': 49496, 'num_clusters_only_not_identified': 249291, 'num_clusters_single_sequence': 109621, 'num_clu

In [52]:
analize_model_clusters(models[1]['clusters'], models[1]['identifications'], ANALYSIS_OUTPUT_FOLDER, spectra_files, all_ids, 1)


Filename root=embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p100.000000
Cluster identification file not found! /media/eduseiti/data_storage_1TB/unicamp/clustering_linfeng_sample_pvalues/identifications/identifications_q0.001_transformer_64_double_n_pair_pvalue_0.6/embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p100.000000_identifications.tsv

Filename root=embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p64.506800
{'unique_and_cluster_pure_count': 3974, 'percent_total_unique': 0.028712835518948016, 'unique_count': 138405, 'percent_total_sequences_clusters': 0.7390034492701003, 'num_sequences_clusters': 187286, 'num_clusters_only_not_identified': 5555, 'num_clusters_single_sequence': 15419, 'num_clusters_multiple_sequences': 50727, 'percent_total_clusters': 0.7074796725289745, 'num_total_clusters': 71701, 'num_not_identified_spectra': 253715, 'percent_total_spectra': 0.27815174325162856, 'num_total_spectra': 912146, 'cluster':

{'unique_and_cluster_pure_count': 64149, 'percent_total_unique': 0.519055248082338, 'unique_count': 123588, 'percent_total_sequences_clusters': 0.65988915348718, 'num_sequences_clusters': 187286, 'num_clusters_only_not_identified': 89054, 'num_clusters_single_sequence': 255244, 'num_clusters_multiple_sequences': 43423, 'percent_total_clusters': 0.11199548128680159, 'num_total_clusters': 387721, 'num_not_identified_spectra': 253715, 'percent_total_spectra': 0.27815174325162856, 'num_total_spectra': 912146, 'cluster': 'embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p95.897800.tsv'}

Filename root=embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p96.000000
{'unique_and_cluster_pure_count': 64713, 'percent_total_unique': 0.5250505066896008, 'unique_count': 123251, 'percent_total_sequences_clusters': 0.6580897664534455, 'num_sequences_clusters': 187286, 'num_clusters_only_not_identified': 90676, 'num_clusters_single_sequence': 259310, 'num_clusters_mult

In [42]:
analize_model_clusters(models[1]['clusters'], models[1]['identifications'], ANALYSIS_OUTPUT_FOLDER, spectra_files, all_ids, 0.01)


Filename root=embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p100.000000
Cluster identification file not found! /media/eduseiti/data_storage_1TB/unicamp/clustering_linfeng_sample_pvalues/identifications/identifications_q0.001_transformer_64_double_n_pair_pvalue_0.6/embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p100.000000_identifications.tsv

Filename root=embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p64.506800
{'unique_and_cluster_pure_count': 1306, 'percent_total_unique': 0.0381325002189845, 'unique_count': 34249, 'percent_total_sequences_clusters': 0.6919549054469049, 'num_sequences_clusters': 49496, 'num_clusters_only_not_identified': 32427, 'num_clusters_single_sequence': 20904, 'num_clusters_multiple_sequences': 18370, 'percent_total_clusters': 0.25620284235924184, 'num_total_clusters': 71701, 'num_not_identified_spectra': 601258, 'percent_total_spectra': 0.6591685980095292, 'num_total_spectra': 912146, 'cluster': 'e

{'unique_and_cluster_pure_count': 20139, 'percent_total_unique': 0.7876950756834983, 'unique_count': 25567, 'percent_total_sequences_clusters': 0.5165467916599321, 'num_sequences_clusters': 49496, 'num_clusters_only_not_identified': 274617, 'num_clusters_single_sequence': 117282, 'num_clusters_multiple_sequences': 1013, 'percent_total_clusters': 0.0025781854461049804, 'num_total_clusters': 392912, 'num_not_identified_spectra': 601258, 'percent_total_spectra': 0.6591685980095292, 'num_total_spectra': 912146, 'cluster': 'embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p96.000000.tsv'}

Filename root=embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p97.000000
{'unique_and_cluster_pure_count': 19025, 'percent_total_unique': 0.8221694036300777, 'unique_count': 23140, 'percent_total_sequences_clusters': 0.46751252626474865, 'num_sequences_clusters': 49496, 'num_clusters_only_not_identified': 315998, 'num_clusters_single_sequence': 135436, 'num_clusters_mu

In [43]:
analize_model_clusters(models[2]['clusters'], models[2]['identifications'], ANALYSIS_OUTPUT_FOLDER, spectra_files, all_ids, 0.01)


Filename root=sample_maracluster.clusters_p10
{'unique_and_cluster_pure_count': 23245, 'percent_total_unique': 0.6876201745303949, 'unique_count': 33805, 'percent_total_sequences_clusters': 0.6829844835946339, 'num_sequences_clusters': 49496, 'num_clusters_only_not_identified': 195959, 'num_clusters_single_sequence': 71866, 'num_clusters_multiple_sequences': 2128, 'percent_total_clusters': 0.007882853681937226, 'num_total_clusters': 269953, 'num_not_identified_spectra': 601258, 'percent_total_spectra': 0.6591685980095292, 'num_total_spectra': 912146, 'cluster': 'sample_maracluster.clusters_p10.tsv'}

Filename root=sample_maracluster.clusters_p15
{'unique_and_cluster_pure_count': 23951, 'percent_total_unique': 0.7626249761192129, 'unique_count': 31406, 'percent_total_sequences_clusters': 0.6345159204784225, 'num_sequences_clusters': 49496, 'num_clusters_only_not_identified': 247077, 'num_clusters_single_sequence': 82609, 'num_clusters_multiple_sequences': 1341, 'percent_total_clusters'

### Analyze some specific clusterizations

In [46]:
results, sequences_purity, cluster_ids = analize_cluster("/media/eduseiti/data_storage_1TB/unicamp/clustering_linfeng_sample_pvalues/clusters/clusters_0.1_embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6/embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p94.880000.tsv", 
                                                         "/media/eduseiti/data_storage_1TB/unicamp/clustering_linfeng_sample_pvalues/identifications/identifications_q0.001_transformer_64_double_n_pair_pvalue_0.6/embeddings_q0.001_transformer_64_double_n_pair_pvalue_0.6.clusters_p94.880000_identifications.tsv", 
                                                         spectra_files, all_ids[all_ids['percolator q-value'] < 0.01], 0.01, None, save_data=False)

In [47]:
results

{'unique_and_cluster_pure_count': 20942,
 'percent_total_unique': 0.7526054768921153,
 'unique_count': 27826,
 'percent_total_sequences_clusters': 0.5621868433812833,
 'num_sequences_clusters': 49496,
 'num_clusters_only_not_identified': 238822,
 'num_clusters_single_sequence': 103278,
 'num_clusters_multiple_sequences': 1425,
 'percent_total_clusters': 0.004148169711083618,
 'num_total_clusters': 343525,
 'num_not_identified_spectra': 601258,
 'percent_total_spectra': 0.6591685980095292,
 'num_total_spectra': 912146}

In [48]:
sequences_purity

{'not identified': array([     1,      2,      3, ..., 343523, 343524, 343525]),
 'E[229.16]THCHQHK[229.16]': array([16]),
 'H[229.16]SNSEHK[229.16]': array([120572, 169333, 252740, 310681]),
 'N[229.16]AHSIHQR': array([ 96146, 135468, 149078, 169335]),
 'FHHTIGGSR': array([    26,   7347, 168809]),
 'EFVHFALYNK': array([169338]),
 'V[229.16]VHQNHGER': array([163299, 169339, 333229]),
 'S[229.16]AHQVAR': array([    33,   7329,  14393, 173174]),
 'A[229.16]LEHHR': array([    45, 163304]),
 'E[229.16]LNHTQQLPDCR': array([49]),
 'E[229.16]AQGQQQHHK[229.16]': array([    65,  96196, 333241]),
 'A[229.16]NMQHR': array([169360]),
 'S[229.16]SGQHQNDK[229.16]': array([    74,    106,  33873,  96221, 147812]),
 'HAVSEGTK[229.16]': array([    77,  33808,  33952,  33975,  34156,  40459,  46032,  80355,
        147800, 153928, 185845, 341617]),
 'A[229.16]HATGAGPAGR': array([    95, 310715]),
 'H[229.16]PHVANPR': array([103]),
 'A[229.16]SAPYNHHGSR': array([   105,  96250, 192491]),
 'A[229.16]LM[1

In [49]:
cluster_ids[171]

{'spectra': [{'sequence': 'Q[229.16]NPSHTDQHTK[229.16]',
   'q-value': 0.00062050733,
   'file': 0,
   'scan': 295},
  {'sequence': 'Q[229.16]NPSHTDQHTK[229.16]',
   'q-value': 0.00011133688,
   'file': 0,
   'scan': 322},
  {'sequence': 'Q[229.16]NPSHTDQHTK[229.16]',
   'q-value': 0.0018805597,
   'file': 5,
   'scan': 214}],
 'sequences': ['Q[229.16]NPSHTDQHTK[229.16]']}

In [50]:
cluster_ids[169409]

{'spectra': [{'sequence': 'Q[229.16]NPSHTDQHTK[229.16]',
   'q-value': 0.00017634672,
   'file': 0,
   'scan': 318}],
 'sequences': ['Q[229.16]NPSHTDQHTK[229.16]']}

In [55]:
print("Cluster 155:\n{}\n".format(cluster_ids[155]))
print("Cluster 157:\n{}\n".format(cluster_ids[157]))
print("Cluster 252808:\n{}".format(cluster_ids[252808]))

Cluster 155:
{'spectra': [{'sequence': 'H[229.16]STGDTK[229.16]', 'q-value': 0.00053588175, 'file': 0, 'scan': 266}, {'sequence': 'H[229.16]STGDTK[229.16]', 'q-value': 0.0043742855, 'file': 5, 'scan': 208}], 'sequences': ['H[229.16]STGDTK[229.16]']}

Cluster 157:
{'spectra': [{'sequence': 'H[229.16]STGDTK[229.16]', 'q-value': 0.00065323003, 'file': 0, 'scan': 268}, {'sequence': 'H[229.16]STGDTK[229.16]', 'q-value': 0.00053588175, 'file': 5, 'scan': 209}, {'sequence': 'H[229.16]STGDTK[229.16]', 'q-value': 0.00032879531999999997, 'file': 28, 'scan': 349}], 'sequences': ['H[229.16]STGDTK[229.16]']}

Cluster 252808:
{'spectra': [{'sequence': 'H[229.16]STGDTK[229.16]', 'q-value': 0.00010701331, 'file': 28, 'scan': 356}], 'sequences': ['H[229.16]STGDTK[229.16]']}


In [26]:
sequences_count = []

for key, cluster in cluster_ids.items():
    
    if "not identified" in cluster['sequences']:
        sequences_count.append(len(cluster['sequences']) - 1)
    else:
        sequences_count.append(len(cluster['sequences']))

In [28]:
len(sequences_count)

308932

In [29]:
len(cluster_ids)

308932

In [30]:
np.mean(sequences_count)

1.0640982481581707

In [31]:
sequences_count = np.array(sequences_count)

In [33]:
len(sequences_count[sequences_count > 1])

51081

In [34]:
np.median(sequences_count)

1.0

In [36]:
len(sequences_count[sequences_count == 0])

64375